In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2023-07-28 09:13:55--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  28.7MB/s    in 2.3s    

2023-07-28 09:13:57 (28.7 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import os   #provides functions for interacting with the operating system
import zipfile

#unzipping files
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

### Defining sub directories

In [3]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

#directory with training cat images
train_cat_dir = os.path.join(train_dir, 'cats')

#directory with training dog images
train_dog_dir = os.path.join(train_dir, 'dogs')

#directory with validation cat images
validation_cat_dir = os.path.join(validation_dir, 'cats')

#directory with training cat images
validation_dog_dir = os.path.join(validation_dir, 'dogs')

Total no of cats and dogs images in the train and validation directories.

In [6]:
print('total training cat images:', len(os.listdir(train_cat_dir)))
print('total training dog images:', len(os.listdir(train_dog_dir)))
print('total validation cat images:', len(os.listdir(validation_cat_dir)))
print('total validation dog images:', len(os.listdir(validation_dog_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500


Image Augmentation

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

Training and Validation Sets

In [8]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (224, 224)
)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (224, 224)
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Loading the base model

we will be only making changes to the final layer. That is because this is a binary classfication problem.

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(
    input_shape = (224, 224, 3), # defining the shape of the input image.
    include_top = False, # Keep the last fully connected layer
    weights = 'imagenet'
)


58889256/58889256 [==============================] - 2s 0us/step


Since no need of training all the layers, we are making them non-trainable

In [11]:
for layer in base_model.layers:
  layer.trainable = False